In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import functools as ft
import multiprocessing as mp
import sys

In [3]:
%matplotlib inline

In [4]:
from hgflask import app
import hgflask
from flask import jsonify, request

In [5]:
import hgtiles

processes = {}

### HiGlass Server

In [76]:
hgflask.TILESETS = [{  
    'uuid': "abc",
    'filetype': "bigwig",
    'datatype': "vector",
    'url': 'http://hgdownload.cse.ucsc.edu/goldenPath/hg19/encodeDCC/wgEncodeSydhTfbs/wgEncodeSydhTfbsGm12878Ctcfsc15914c20StdSig.bigWig',
    'private': False,
    'name': "blabla",
    'coordSystem': "hg19",
    'coordSystem2': "hg19",
    'created': "2017-02-02T21:10:56.265085Z",
    },
    {
        'uuid': 'a',
        'filetype': 'cooler',
        'datatype': 'matrix',
        'filepath': '/Users/pete/Dropbox/paper-data/phillippy-data/out.mcool',
    }
]

In [77]:
import slugid

for puid in processes:
    processes[puid].terminate()

processes = {}
print(hgflask.TILESETS)
# we're going to assign a uuid to each server process so that if anything
# goes wrong, the variable referencing the process doesn't get lost
uuid = slugid.nice()
processes[uuid] = mp.Process(target=ft.partial(app.run, debug=True, port=4006, host='0.0.0.0', use_reloader=False))
processes[uuid].start()

[{'uuid': 'abc', 'filetype': 'bigwig', 'datatype': 'vector', 'url': 'http://hgdownload.cse.ucsc.edu/goldenPath/hg19/encodeDCC/wgEncodeSydhTfbs/wgEncodeSydhTfbsGm12878Ctcfsc15914c20StdSig.bigWig', 'private': False, 'name': 'blabla', 'coordSystem': 'hg19', 'coordSystem2': 'hg19', 'created': '2017-02-02T21:10:56.265085Z'}, {'uuid': 'a', 'filetype': 'cooler', 'datatype': 'matrix', 'filepath': '/Users/pete/Dropbox/paper-data/phillippy-data/out.mcool'}]


 * Running on http://0.0.0.0:4006/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Feb/2018 15:32:21] "GET /api/v1/tiles/?d=a.0.0.0 HTTP/1.1" 200 -


In [78]:
!curl localhost:4006/api/v1/tiles/?d=a.0.0.0

{
  "a.0.0.0.default": {


    "dense": "...

...", 
    "dtype": "float32", 
    "max_value": 197999.0, 
    "min_value": 0.0
  }
}
